### Wprowadzenie - klasyfikacja

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

x, y = make_blobs(n_samples=300, n_features=2, random_state=12, centers=3, shuffle=True, cluster_std=1.5)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0, shuffle=True, stratify=y)

knn = KNeighborsClassifier(n_neighbors=5)
svml = SVC(kernel='linear', C=1.0, random_state=0)
svmrbf = SVC(kernel='rbf', C=1.0, random_state=0)

for i, model in enumerate([knn, svml, svmrbf]): 

    # Train the model using the training sets
    model.fit(x_train, y_train)

    # Make predictions using the testing set
    y_pred = model.predict(x_test)

    # The mean squared error
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

    # Plot outputs
    axes[i].scatter(x_test[:,0], x_test[:,1], c=y_test, marker='o')
    axes[i].scatter(x_test[:,0], x_test[:,1], c=y_pred, marker='.')

    axes[i].set_xticks(())
    axes[i].set_yticks(())

    axes[i].set_xlabel("Dimension 1")
    axes[0].set_ylabel("Dimension 2")

    axes[i].set_title(f"Accuracy = {accuracy_score(y_test, y_pred):.2f}")

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

DATASET_PATH = "Titanic.csv"

In [ ]:
# Wczytanie datasetu Breast Cancer
dataset = pd.read_csv(DATASET_PATH)

In [ ]:
# Informacje o zestawie danych
dataset.info()

In [ ]:
DATASET_FEATURE_LABELS = {
    "PassengerId": "Passenger ID",
    "Survived": "Survival",
    "Pclass": "Class",
    "Name": "Name",
    "Sex": "Sex",
    "Age": "Age",
    "SibSp": "Siblings/Spouses",
    "Parch": "Parents/Children",
    "Ticket": "Ticket Number",
    "Fare": "Passenger Fare",
    "Cabin": "Cabin",
    "Embarked": "Port of embarkation",
}

In [ ]:
# Zmiana nazwy kolumn
dataset = dataset.rename(columns=DATASET_FEATURE_LABELS)
dataset.info()

In [ ]:
# Usunięcie zbędnych kolumn
dataset = dataset.drop(columns=['Passenger ID', 'Name', 'Ticket Number', 'Cabin'])

### Brakujące dane

In [ ]:
# Znalezienie brakujących danych
dataset.isnull().any()
# W rzędzie jest brakująca wartość
isnull = dataset.isnull().any(axis=1)
print(np.count_nonzero(isnull),"brakujących wartości.")
# Usunięcie rzędu
dataset = dataset.drop(np.asarray(isnull).nonzero()[0].tolist() ,axis=0)

In [ ]:
# Podgląd pierwszych 10 rzędów
dataset.head(10)

### Wydzielenie zmiennej zależnej, liczba klas

In [ ]:
# Wydzielenie zmiennej zależnej (Y)
x,y = dataset.drop(columns=["Survival"]), dataset["Survival"]

In [ ]:
# Określenie liczby klas
counts = y.value_counts()
counts

In [ ]:
# Współczynnik niezbalansowania
c_max, c_min = max(counts.values[0],counts.values[1]),min(counts.values[0],counts.values[1])
imbalance = c_max/c_min

print(f"Współczynnik niezbalansowania: {c_max} / {c_min} = {imbalance:.1f}")

### EDA

In [ ]:
# Podstawowa analiza statystyczna
x.describe()

In [ ]:
# Histogramy zmiennych niezależnych
axes = x.hist(figsize=(10,8), bins=20, edgecolor='black', grid=False)
# Dodatkowe formatowanie
axes = axes.flatten()
axes[0].set_xticks([1,2,3])

In [ ]:
# Analiza BOXPLOT

import seaborn as sns
import matplotlib.pyplot as plt


fig, axes = plt.subplots(1,3,figsize=(12,4))
sns.boxplot(x=y, y=x["Age"], ax=axes[0])
sns.boxplot(x=y, y=x["Age"], hue=x["Class"], ax=axes[1])
sns.boxplot(x=x["Sex"], y=x["Age"], hue=y, ax=axes[2])
plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1,3,figsize=(12,4))
sns.boxplot(x=y, y=x["Passenger Fare"], ax=axes[0])
sns.boxplot(x=y, y=x["Passenger Fare"], hue=x["Class"], ax=axes[1])
sns.boxplot(x=x["Class"], y=x["Passenger Fare"], ax=axes[2])
plt.tight_layout()
plt.show()


In [ ]:
# Analiza korelacji pomiędzy zmiennymi

import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(x.select_dtypes(exclude='object').corr(), ax=ax, annot=True, fmt=".2f", cmap="coolwarm", center=0)

### Modele klasyfikacji - SVM, kNN

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder 
from sklearn.compose import make_column_transformer


col_categorical = x.select_dtypes(include='object').columns
col_numerical = x.select_dtypes(exclude='object').columns

col_transformer = make_column_transformer(
    (StandardScaler(), col_numerical),
    (OrdinalEncoder(), col_categorical)
)

from  sklearn.preprocessing import LabelEncoder

# Enkodowanie zmiennej zależnej
le = LabelEncoder()
y = le.fit_transform(y)
y_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
y_name_mapping

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

svc = Pipeline([
    ('col_transformer', col_transformer),
    ('svc', SVC(kernel="linear", C=1.0)),
])

knn = Pipeline([
    ('col_transformer', col_transformer),
    ('knn', KNeighborsClassifier(n_neighbors=3)),
])


### Walidacja krzyżowa dla domyślnych hiperparametrów

- Jaki wpływ na model może mieć podział danych wejściowych na zestaw treningowy i walidacyjny?
- Jak zachować się w przypadku małych zestawów danych?

-> **Walidacja krzyżowa**:
Zastosujemy walidację krzyżową aby porównać dwa modele - kNN oraz SVC.

In [ ]:
from sklearn.model_selection import cross_validate

SCORING = ('accuracy', 'recall', 'precision', 'f1')

results = cross_validate(svc, x, y, cv=5, scoring=SCORING)

print("=====\nSVC:\n=====")
for key in results.keys():
    print(f"{key}: {results[key].mean():.3f}")

results = cross_validate(knn, x, y, cv=5, scoring=SCORING)

print("=====\nk-NN:\n=====")
for key in results.keys():
    print(f"{key}: {results[key].mean():.3f}")

### Dobór hiperparametrów metodą GridSearch

In [ ]:
# Listowanie parametrów SVC
svc['svc'].get_params()

In [ ]:
# Listowanie parametrów k-NN
knn['knn'].get_params()

In [ ]:
# Tuning hiperparametrów k-NN z wykorzystaniem GridSearchCV 
from sklearn.model_selection import GridSearchCV

parameters = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11],
    'knn__metric': ['euclidean', 'cosine', 'cityblock', 'l1', 'l2', 'nan_euclidean'],
}
grid_search = GridSearchCV(knn, parameters)
grid_search.fit(x, y)

# Zastosowanie hiperparametrów
for param, val in grid_search.best_params_.items():
    knn.set_params(**{param: val})

grid_search.best_params_

In [ ]:
# Tuning hiperparametrów SVC z wykorzystaniem GridSearchCV 
from sklearn.model_selection import GridSearchCV

parameters = {
    'svc__C': list(np.arange(0.1, 2.1, 0.1)),
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}
grid_search = GridSearchCV(svc, parameters)
grid_search.fit(x, y)

# Zastosowanie hiperparametrów
for param, val in grid_search.best_params_.items():
    svc.set_params(**{param: val})

grid_search.best_params_

In [ ]:
results = cross_validate(svc, x, y, cv=5, scoring=SCORING)

print("=====\nSVC:\n=====")
for key in results.keys():
    print(f"{key}: {results[key].mean():.3f}")

results = cross_validate(knn, x, y, cv=5, scoring=SCORING)

print("=====\nk-NN:\n=====")
for key in results.keys():
    print(f"{key}: {results[key].mean():.3f}")

### DODATEK: Porównanie klasyfikatorów
https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [ ]:
# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    RandomForestClassifier(
        max_depth=5, n_estimators=10, max_features=1, random_state=42
    ),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(algorithm="SAMME", random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

X, y = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1
)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1),
    linearly_separable,
]

figure = plt.figure(figsize=(27, 9))
i = 1
# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42
    )

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(["#FF0000", "#0000FF"])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    if ds_cnt == 0:
        ax.set_title("Input data")
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k")
    # Plot the testing points
    ax.scatter(
        X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6, edgecolors="k"
    )
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)

        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        DecisionBoundaryDisplay.from_estimator(
            clf, X, cmap=cm, alpha=0.8, ax=ax, eps=0.5
        )

        # Plot the training points
        ax.scatter(
            X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
        )
        # Plot the testing points
        ax.scatter(
            X_test[:, 0],
            X_test[:, 1],
            c=y_test,
            cmap=cm_bright,
            edgecolors="k",
            alpha=0.6,
        )

        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name)
        ax.text(
            x_max - 0.3,
            y_min + 0.3,
            ("%.2f" % score).lstrip("0"),
            size=15,
            horizontalalignment="right",
        )
        i += 1

plt.tight_layout()
plt.show()